In [1]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import sys

sys.path.append("../python")

import utils


/home/musella/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
max_words = 1600
glove_dim = 25
hash_dim = 5

In [4]:
words_embed = utils.load_glove_embedding(utils.glove_embedding_path(25))
hash_embed = utils.load_glove_embedding('../data/models/hastags/hash_vectors.d'+str(hash_dim)+'.txt')

In [16]:
df = pd.read_hdf("../data/preproc/tweets.hd5")

tk = Tokenizer(num_words=max_words,
               filters="", # already applied
               lower=True,
               split=" ")
tk.fit_on_texts(df["preproc_text"]) 

In [17]:
reload(utils)
words_embed_mtx, words_unk = utils.fill_embedding_matrix(tk.word_index,words_embed,max_words,glove_dim)
hash_embed_mtx, hash_unk = utils.fill_embedding_matrix(tk.word_index,hash_embed,max_words,hash_dim)


In [18]:
unk = { word: tk.word_index[word] for word in set(words_unk).intersection(set(hash_unk)) }
unk

{'#mercerchats': 1233,
 '#opensource': 1512,
 '#payments': 1398,
 '<stop>': 3,
 '<trunc>': 4,
 'agritech': 1322,
 'bottou': 1339,
 'chatbot': 888,
 'chatbots': 1097,
 'convolutional': 623,
 'datax': 700,
 'daysofmlcode': 1227,
 'deeplearn007': 254,
 'interpretability': 1543,
 'ipfconline1': 970,
 'jimmarous': 620,
 'kirkdborne': 1284,
 'machinelearning': 1041,
 'mikequindazzi': 265,
 'neurips': 325,
 'nodexl': 583,
 'parkinson’s': 1329,
 'pineau': 1065,
 'preprint': 1527,
 'prioirites': 1171,
 'probabilistic': 1013,
 'pulipaka': 1582,
 'reproducibility': 1239,
 'selectscience': 1438,
 'spirosmargaris': 575,
 'tencent’s': 1357,
 'tensorflow': 492,
 'vanloon': 174,
 'variational': 1020,
 'verisk': 1285,
 'whova': 1133,
 'world’s': 1274,
 'yape': 902}

In [19]:
stop_mtx = np.zeros( (max_words+1, 2) )

stop_mtx[unk.pop("<trunc>"),0] = 1.
stop_mtx[unk.pop("<stop>"),1] = 1.

In [20]:
unk_idx = list(unk.values())
unk

{'#mercerchats': 1233,
 '#opensource': 1512,
 '#payments': 1398,
 'agritech': 1322,
 'bottou': 1339,
 'chatbot': 888,
 'chatbots': 1097,
 'convolutional': 623,
 'datax': 700,
 'daysofmlcode': 1227,
 'deeplearn007': 254,
 'interpretability': 1543,
 'ipfconline1': 970,
 'jimmarous': 620,
 'kirkdborne': 1284,
 'machinelearning': 1041,
 'mikequindazzi': 265,
 'neurips': 325,
 'nodexl': 583,
 'parkinson’s': 1329,
 'pineau': 1065,
 'preprint': 1527,
 'prioirites': 1171,
 'probabilistic': 1013,
 'pulipaka': 1582,
 'reproducibility': 1239,
 'selectscience': 1438,
 'spirosmargaris': 575,
 'tencent’s': 1357,
 'tensorflow': 492,
 'vanloon': 174,
 'variational': 1020,
 'verisk': 1285,
 'whova': 1133,
 'world’s': 1274,
 'yape': 902}

In [21]:
np.isnan(words_embed_mtx).sum()

0

In [22]:
np.isnan(hash_embed_mtx).sum()

0

In [23]:
# ! mkdir ../data/models/sequences

import pickle
import json

with open("../data/models/sequences/info.json","w+") as out:
    save = dict(max_words=max_words,unk_idx=unk_idx)
    out.write(json.dumps(save))
    
with open('../data/models/sequences/tokenizer.pkl','wb+') as out:
    out.write( pickle.dumps(tk) )
    
np.save('../data/models/sequences/embed_mtx.npy',words_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',hash_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',stop_mtx)

    


In [24]:
from sklearn.model_selection import train_test_split

df["sequences"] = tk.texts_to_sequences(df["preproc_text"])

df.to_hdf("../data/preproc/sequences.hd5","squences",columns=["sequences"],mode="w")

# index = np.arange(df.index.shape[0]).astype(np.int)
# train_idx, test_idx = train_test_split(index,test_size=0.2,random_state=123456)
# df_train = df.iloc[train_idx]
# df_test = df.iloc[test_idx]
# df_train.to_hdf("../data/preproc/sequences.hd5","train",columns=["sequences"])
# df_test.to_hdf("../data/preproc/sequences.hd5","test",columns=["sequences"])


/home/musella/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['lang', 'text', 'user/name', 'preproc_text', 'sequences']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [14]:
# ?tk.texts_to_sequences